### Adaptive RAG documentation

Has many RAG application usage examples

https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag/

I use joplin to store my notes and have exporting one notebook in markdown format present in path - "D:\Downloads\k8s-joplin"
Now, I have been focussing on creating headings for each paragraphas and so wil be using **MarkdownHeaderTextSplitter**.

Had I been using some other files like txt then would have preferred **RecursiveCharacterTextSplitter**

For markdown loading, we use markdown loader - https://python.langchain.com/v0.2/docs/how_to/document_loader_markdown/

Note - These are local notes being loaded and don't have a real folder in this repo, please refer to this code to **load markdown files from a folder for RAG**

In [1]:
import os

directory_path = "./k8s-joplin/Kubernetes/" # Kubernetes folder has md files and _resources folder has corresponding images
files_list = os.listdir(directory_path)
files_list[0]


'AdmissionregistrationApi.md'

### For each of files, split via headings and then use loader to load these chunks to pages

reference code -https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.ipynb

### First remove the markdown syntax using Unstructured markdown reader and then chunk it via recursive text splitter

In [2]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(directory_path, glob="./*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)
documents = loader.load()

100%|██████████| 731/731 [00:58<00:00, 12.50it/s]


In [7]:
# [d.metadata['source'] for d in documents]

In [3]:
len(documents)

731

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size=1000
chunk_overlap=200

# do recursive split on big chunks of splits on markdown splits as well
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

splits = text_splitter.split_documents(documents=documents)

In [5]:
len(splits)

9303

### Start from here

In [6]:
import pickle

# with open('splits.pkl', 'wb') as f:
#     pickle.dump(splits, f)

with open('splits.pkl', 'rb') as f:
    splits = pickle.load(f)

In [7]:
len(splits)

9303

### Create embeddings and load to vector database chromaDB

In [1]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_chroma import Chroma

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
persist_dir = "chromadb"
# db = Chroma.from_documents(documents=splits, embedding=embedding_function, persist_directory=persist_dir)

C:\Users\VARUN ARORA\AppData\Local\Temp\ipykernel_4184\2451541155.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
d:\VS Code\python\GenAI-Cookbook\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\VS Code\python\GenAI-Cookbook\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by defau

In [ ]:
newdb = Chroma(persist_directory="chromadb", embedding_function=embedding_function)